# Translating English Sentences into Propositional Logic Statements

The following notebook was adapted from Peter Norvig (https://github.com/norvig/). The intention here is to remind us what Propositional Logic statements look like, how to construct them, and what the common mathematical symbols mean.

## Introduction

A *proposition* is a statement about the world that is either true or false. Propositions are typically represented by uppercase Latin letters; the letter P is commonly used for the first proposition in a statement, Q the second, and so on. Proposition can be cominbed with a set of basic rules:

- Every propositional symbol and truth value is a sentence **P, Q**
- The negation (not) of a sentence is a sentence: **¬P**, **¬false**
- The conjunction (and) of two sentences is a sentence: **P ⋀ ¬Q**
- The disjunction (or) of two sentences is a sentence: **P ⋁ ¬Q**
- The implication (implies) of two sentences is a sentence: **P ⇒ ¬Q**
- The equivalence of two sentences is a sentence: **P ⋁ Q ≡ R**

A common exercise in learning logic is to turn an English sentence like this:

> *Sieglinde will survive, and either her son will gain the Ring and Wotan’s plan will be fulfilled or else Valhalla will be destroyed.*

into a formal Propositional Logic statement: 

    P ⋀ ((Q ⋀ R) ⋁ S)
    
along with definitions of the propositions:

    P: Sieglinde will survive
    Q: Sieglinde’s son will gain the Ring
    R: Wotan’s plan will be fulfilled
    S: Valhalla will be destroyed

For some sentences, it takes detailed knowledge to get a good translation. The following two sentences are ambiguous, with different prefered interpretations, and translating them correctly requires knowledge of eating habits:

    I will eat salad or I will eat bread and I will eat butter.     P ∨ (Q ⋀ R)
    I will eat salad or I will eat soup  and I will eat ice cream. (P ∨ Q) ⋀ R

But for many sentences, the translation process is automatic, with no special knowledge required. The following is programmme to handle these easy sentences. The programme is based on the idea of a series of translation rules of the form:

    Rule('{P} ⇒ {Q}', 'if {P} then {Q}', 'if {P}, {Q}')
    
which means that the logic translation will have the form `'P ⇒ Q'`, whenever the English sentence has either the form `'if P then Q'` or  `'if P, Q'`, where `P` and `Q` can match any non-empty subsequence of characters.  Whatever matches `P` and `Q` will be recursively processed by the rules. The rules are in order&mdash;top to bottom, left to right, and the first rule that matches in that order will be accepted, no matter what, so be sure you order your rules carefully. One guideline adhered to is to put all the rules that start with a keyword (like `'if'` or `'neither'`) before the rules that start with a variable (like `'{P}'`); that way you avoid accidently having a keyword swallowed up inside a `'{P}'`.

Notice that given the sentence "*Sieglinde will survive*", the program should make up a new propositional symbol, `P`, and record the fact that `P` refers to "*Sieglinde will survive*". But the negative sentence "*Sieglinde will not survive*", should be translated as `～P`, where again `P` is  "*Sieglinde will survive*". So to fully specify the translation process, we need to define both `rules` and `negations`. (We do that using [regular expressions](https://docs.python.org/3.8/library/re.html), which can sometimes be confusing.)

## The Programme

First the function to define a rule (and some auxiliary functions):

In [ ]:
import re

def Rule(output, *patterns):
    "A rule that produces `output` if the entire input matches any one of the `patterns`." 
    return (output, [name_group(pat) + '$' for pat in patterns])

def name_group(pat):
    "Replace '{Q}' with '(?P<Q>.+?)', which means 'match 1 or more characters, and call it Q'"
    return re.sub('{(.)}', r'(?P<\1>.+?)', pat)
            
def word(w):
    "Return a regex that matches w as a complete word (not letters inside a word)."
    return r'\b' + w + r'\b' # '\b' matches at word boundary

And now the actual rules. If you have a sentence that is not translated correctly by this program, you can  augment these rules to handle your sentence.

In [ ]:
rules = [
    Rule('{P} ⇒ {Q}',         'if {P} then {Q}', 'if {P}, {Q}'),
    Rule('{P} ⋁ {Q}',          'either {P} or else {Q}', 'either {P} or {Q}'),
    Rule('{P} ⋀ {Q}',          'both {P} and {Q}'),
    Rule('～{P} ⋀ ～{Q}',       'neither {P} nor {Q}'),
    Rule('～{A}{P} ⋀ ～{A}{Q}', '{A} neither {P} nor {Q}'), # The Kaiser neither ...
    Rule('～{Q} ⇒ {P}',        '{P} unless {Q}'),
    Rule('{P} ⇒ {Q}',          '{Q} provided that {P}', '{Q} whenever {P}', '{P} implies {Q}',
                                '{P} therefore {Q}', '{Q}, if {P}', '{Q} if {P}', '{P} only if {Q}'),
    Rule('{P} ⋀ {Q}','{P} and {Q}', '{P} but {Q}'),
    Rule('{P} ⋁ {Q}',          '{P} or else {Q}', '{P} or {Q}'),
    ]

negations = [
    (word("not"), ""),
    (word("cannot"), "can"),
    (word("can't"), "can"),
    (word("won't"), "will"),
    (word("ain't"), "is"),
    ("n't", ""), # matches as part of a word: didn't, couldn't, etc.
    ]

Now the mechanism to process these rules. Note that `defs` is a dict of definitions of propositional symbols: `{P: 'english'}`. The three `match_*` functions return two values: the translation of a sentence, and a dict of defintions.

In [ ]:
def match_rules(sentence, rules, defs):
    """Match sentence against all the rules, accepting the first match; or else make it an atomic proposition.
    Return two values: the Logic translation and a dict of {P: 'english'} definitions."""
    sentence = clean(sentence)
    for rule in rules:
        result = match_rule(sentence, rule, defs)
        if result: 
            return result
    return match_atomic_proposition(sentence, negations, defs)
        
def match_rule(sentence, rule, defs):
    "Match a single rule, returning the logic translation and the dict of definitions if the match succeeds."
    output, patterns = rule
    for pat in patterns:
        match = re.match(pat, sentence, flags=re.I)
        if match:
            groups = match.groupdict()
            for P in sorted(groups): # Recursively apply rules to each of the matching groups
                groups[P] = match_rules(groups[P], rules, defs)[0]
            return '(' + output.format(**groups) + ')', defs
        
def match_atomic_proposition(sentence, negations, defs):
    "No rule matched; sentence is an atom. Add new proposition to defs. Handle negation."
    polarity = ''
    for (neg, pos) in negations:
        (sentence, n) = re.subn(neg, pos, sentence, flags=re.I)
        polarity += n * '～'
    sentence = clean(sentence)
    P = proposition_name(sentence, defs)
    defs[P] = sentence
    return polarity + P, defs
    
def proposition_name(sentence, defs, names='PQRSTUVWXYZBCDEFGHJKLMN'):
    "Return the old name for this sentence, if used before, or a new, unused name."
    inverted = {defs[P]: P for P in defs}
    if sentence in inverted:
        return inverted[sentence]                      # Find previously-used name
    else:
        return next(P for P in names if P not in defs) # Use a new unused name
    
def clean(text): 
    "Remove redundant whitespace; handle curly apostrophe and trailing comma/period."
    return ' '.join(text.split()).replace("’", "'").rstrip('.').rstrip(',')

import textwrap

def parse(sentence, width=106):
    "Match the rules against a sentence in text, and print the result."
    if sentence.strip() == "":
        print("Sentence empty.")
        return
    s = clean(sentence)
    logic, defs = match_rules(s, rules, {})
    print(width*'_', '\n' + textwrap.fill(s, width), '\nLogic:', logic)
    for P in sorted(defs):
      print('{}: {}'.format(P, defs[P]))

## Preparation 

You must execute all the cells above for the cells below to work. You can do this one at a time, or use the "Execute Above Cells" button on the code cell below (hover and it appears in the top-right of the cell along with Other Actions...).

## Exercise 1 - Predicting Outputs

The function `parse` will attempt to parse a sentence and print the output. In this exercise, a series of sentences are provided. Read each of the following sentences and note down what you expect the output to be in the comment, then run them to compare the output (uncomment the `parse` function to check the output). You can read the `Rules` defined above to help you. 

### a. "Polkadots and Moonbeams"

In [ ]:
# expected outcome: 
s = "Polkadots and Moonbeams."

#parse(s)

### b. "Should I stay or should I go"

In [ ]:
# expected outcome: 
s = "Should I stay or should I go."

#parse(s)

### c. "Either Danny didn't come to the party or Virgil didn't come to the party"

In [ ]:
# expected outcome: (～P ⋁ ～Q)
s = "Either Danny didn't come to the party or Virgil didn't come to the party."

parse(s)

### d. "If you liked it then you shoulda put a ring on it."

In [ ]:
# expected outcome: 
s = "If you liked it then you shoulda put a ring on it."

#parse(s)

### e. "It don't mean a thing, if it ain't got that swing."

In [ ]:
# expected outcome: 
s = "It don't mean a thing, if it ain't got that swing."

parse(s)

## Exercise 2 - Forming Sentences

In this exercise you are provided with a proposition and should attempt to write a sentence that generates that proposition. Note there are limitations to the script which are detailed below; these restrict the kinds of sentence you can write. You can look at the `Rules` to see what words and phrases match to the conjunctions you need. If you are having difficulty, try to build up from parts of the sentence rather than trying all at once; you can also write sentences with P, Q, etc. then replace these later.

### a. (P ⋀ (Q ⋁ R))

In [ ]:
# expected outcome: (P ⋀ (Q ⋁ R))
s = ""

parse(s)

### b. (P ⋀ (Q ⋀ (R ⋀ (S ⋀ T))))

In [ ]:
# expected outcome: (P ⋀ (Q ⋀ (R ⋀ (S ⋀ T))))
s = ""

parse(s)

### c. (P ⇒ ～Q)

In [ ]:
# expected outcome: (P ⇒ ～Q)
s = ""

parse(s)

### d. (～P ⋀ ～～P)

In [ ]:
# expected outcome: (～P ⋀ ～～P)
s = ""

parse(s)

### e. (P ⇒ (Q ⋀ R))

In [ ]:
# expected outcome: (P ⇒ (Q ⋀ R))
s = ""

parse(s)

### f. ((P ⋀ Q) ⇒ R)

In [ ]:
# expected outcome: (P ⇒ (Q ⋀ R))
s = ""

parse(s)

# Exercise 3 - Extensions



The above propositions are simple sentences, but there are other logics that inlude features that allow us to describe more complex properties. *Predicate* or *first-order logic* (https://en.wikipedia.org/wiki/First-order_logic) is a logic allows us to define quantified variables such that the outcome is determined based on the value of the variable. A **predicate** is a sentence that contains a finite number of variables and becomes a statement when specific values are
substituted for the variables. A classic example is:

    All people are mortal.
    Socrates is a person.
    Therefore, Socrates is mortal.

If `P` is a predicate symbol, then:

    P stands for "is mortal"
    P(x) stands for "x is mortal"
    P(Socrates) stand for "Socrates is mortal"

A **quantifier** is a word that refers to quantities ("some" or "all") and say how many elements of a given predcate are true. There are two quantifiers in first-order logic:

- **Universal quantification**; symbol `'∀'` and read as "for all." This is a general form of `'⋀'` (AND), since the property must hold for every value of the variable. A basic form would be `'∀ P x, x is Q'`, or more commonly when discussing sets, `'∀ x ∈ P, Q(x)'` ("for all x in the set P, Q of x is true").
- **Existential quantification**; symbol `'∃'` and read as "there exists." This is a general form of ⋁ (OR), since the property must hold for at least one value of the variable. A basic form would be `∃ P x, such that x satisfies Q` or `'∃ x ∈ P s.t. Q(x)'` ("there exists x in the set P such that x satisfies Q").

Note that the negation of universal quantification is logically equivalent to finding a counterexample, i.e. "there exists x in the set P such that Q of x is not true." An extension of existential quantification is **unique existential quantification** written as `'∃!'` stating that there is "one an only one" object satisfying a condition. 

### a. "All humans are alive"

Add a rule to parse this sentence. Note the first parameter for a `Rule` object is the "Logic:" output, and the remaining parameters are sentences to match. Note that due to the left-to-right matching implemented, it is not possible to parse compund predicates without further modifications.

In [ ]:
#r = Rule('',  '')
#rules.append(r)

s = "All humans are alive"
parse(s)

### b. "Every cloud has a silver lining"

Redefine your rule to include parse this phrase.

In [ ]:
#r = Rule('', '', '')
#rules.append(r)

s = "Every cloud has a silver lining"
parse(s)

### c. Existential quantification

Add a rule to parse a sentence matching existential quantification, and provide an example. You can use the phrase "Some ..." or select another word. 

In [ ]:
#r = Rule('',  '')
#rules.append(r)

s = "Some foods are tasty"
parse(s)

### d. Unique existential quantification

Add a rule to parse a sentence matching unique existential quantification, and provide an example. 

In [ ]:
#r = Rule('',  '')
#rules.append(r)

s = "There is only one food that I like"
parse(s)

# Limitations

Here are some limitations with the above implementation; there are likely to be many more:

* `Should I stay` *etc.*:<br>questions are not poropositional statements.

* `If I were a carpenter`:<br>doesn't handle modal logic.

* `nothing is better`:<br>doesn't handle quantifiers.

* `Either Wotan will triumph and Valhalla will be saved or else he won't`:<br>gets `'he will'` as one of the propositions, but better would be if that refered back to `'Wotan will triumph'`.

* `Wotan will intervene and cause Siegmund's death`:<br>gets `"cause Siegmund's death"` as a proposition, but better would be `"Wotan will cause Siegmund's death"`.

* `Figaro and Susanna will wed`:<br>gets `"Figaro"` and `"Susanna will wed"` as two separate propositions; this should really be one proposition. 

* `"either Antonio or Figaro pays"`:<br>gets `"Antonio"` as a proposition, but it should be `"Antonio pays"`.

* `If the Kaiser neither prevents`:<br>uses the somewhat bogus propositions `PQ` and `PR`. This should be done in a cleaner way. The problem is the same as the previous problem with Antonio: I don't have a good way to attach the subject of a verb phrase to the multiple parts of the verb/object, when there are multiple parts.